In [21]:
from qiskit import QuantumCircuit

In [22]:
def deutschFunction(case : int):
    """generating a valid deutsch function"""
    #currently not important
    if case not in [1,2,3,4]:
        raise ValueError(f'{case} must be 1,2,3,4')
        
    f = QuantumCircuit(2)
    if case in [2,3]:
        f.cx(0,1)
    if case in [3,4]:
        f.x(1)
    return f

In [23]:
display(deutschFunction(3).draw())

q_0: ──■───────
     ┌─┴─┐┌───┐
q_1: ┤ X ├┤ X ├
     └───┘└───┘

In [77]:
def compile_circuit(function : QuantumCircuit):
    """forms a circuit for use in deutsch algorithm"""
    n = function.num_qubits - 1
    qc = QuantumCircuit(n+1, n)
    qc.x(n)
    qc.h(range(n+1))
    
    qc.barrier()
    qc.compose(function, inplace = True)
    qc.barrier()
    
    qc.h(range(n))
    qc.measure(range(n), range(n))
    return qc

In [78]:
display(compile_circuit(deutschFunction(3)).draw())

┌───┐      ░            ░ ┌───┐┌─┐
q_0: ┤ H ├──────░───■────────░─┤ H ├┤M├
     ├───┤┌───┐ ░ ┌─┴─┐┌───┐ ░ └───┘└╥┘
q_1: ┤ X ├┤ H ├─░─┤ X ├┤ X ├─░───────╫─
     └───┘└───┘ ░ └───┘└───┘ ░       ║ 
c: 1/════════════════════════════════╩═
                                     0

In [79]:
#running the circuit
from qiskit_aer import AerSimulator
def deutsch_algorithm(function : QuantumCircuit):
    """determines if circuit is balanced"""
    qc = compile_circuit(function)
    
    result = AerSimulator().run(qc, shots = 1, memory = True).result()
    measurements = result.get_memory()
    if measurements[0]=='0':
        return "constant"
    return "balanced"

In [88]:
for i in range(1,5):
    f = deutschFunction(i)
    display(i,f.draw())
    display(deutsch_algorithm(f))
    print("-------------")

1

q_0: 
     
q_1:

'constant'

-------------


2

...

-------------


4

q_0: ─────
     ┌───┐
q_1: ┤ X ├
     └───┘

'constant'

-------------
